In [ ]:
import pandas as pd

# 파일 경로
car_path = "../data/raw/서울특별시_행정동별 자동차 등록대수 현황_20220430.csv"
coord_path = "../data/processed/admin_centroids.csv"
grid_path = "../data/processed/grid_system_processed.csv"

# 로딩
df = pd.read_csv(car_path, encoding="cp949")
coords = pd.read_csv(coord_path)
grid = pd.read_csv(grid_path)

In [4]:
import pandas as pd

# 파일 경로
file_path = "../data/raw/서울특별시_행정동별 자동차 등록대수 현황_20220430.csv"

# 데이터 불러오기
df = pd.read_csv(file_path, encoding="cp949")

In [ ]:

df['계'] = pd.to_numeric(df['계'], errors='coerce')

# 현재 전기차 수요
ev_current = df[df['연료'] == '전기'].groupby(['사용본거지 시군구', '읍면동 (행정동)'])['계'].sum().reset_index()
ev_current.rename(columns={'계': 'current_ev_count'}, inplace=True)

# 전체 차량 → 미래 전기차 수요 계산
total = df.groupby(['사용본거지 시군구', '읍면동 (행정동)'])['계'].sum().reset_index()
total.rename(columns={'계': 'total_vehicles'}, inplace=True)
total['future_ev_count'] = total['total_vehicles'] * 0.1 # 전체 10%

# 합치기 (미래 수요 = 현재 전기차 + 미래 예측분)
future_demand = pd.merge(ev_current, total, on=['사용본거지 시군구', '읍면동 (행정동)'], how='outer').fillna(0)
future_demand['adjusted_ev_demand'] = future_demand['current_ev_count'] + future_demand['future_ev_count']


In [14]:
!pip install python-dotenv

In [15]:
import requests
import time
import os
from dotenv import load_dotenv

load_dotenv()

# VWorld API 키 입력
VWORLD_API_KEY = os.getenv('VWORLD') # 👉 본인 API 키로 교체하세요

# 1. '기타' 동 제외
future_demand = future_demand[~future_demand['읍면동 (행정동)'].str.contains("기타")].copy()

# 2. 고유한 주소 목록 생성
area_list = future_demand[['사용본거지 시군구', '읍면동 (행정동)']].drop_duplicates().copy()
area_list['full_address'] = "서울특별시 " + area_list['사용본거지 시군구'] + " " + area_list['읍면동 (행정동)']

# 3. 좌표 요청 함수 정의
def get_coords_vworld(address):
    apiurl = "https://api.vworld.kr/req/address?"
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "parcel",  # 도로명주소: 'road', 지번주소: 'parcel'
        "key": VWORLD_API_KEY
    }
    response = requests.get(apiurl, params=params)
    if response.status_code == 200:
        try:
            res = response.json()
            point = res['response']['result']['point']
            return float(point['y']), float(point['x'])  # 위도, 경도
        except:
            return None, None
    return None, None

# 4. 주소별로 위경도 수집
lat_list, lon_list = [], []
for addr in area_list['full_address']:
    lat, lon = get_coords_vworld(addr)
    lat_list.append(lat)
    lon_list.append(lon)
    print(f"{addr} -> lat: {lat}, lon: {lon}")
    time.sleep(0.3)  # 너무 빠른 요청 방지

# 5. future_demand에 좌표 병합
area_list['lat'] = lat_list
area_list['lon'] = lon_list

# 좌표 병합
coords = area_list[['사용본거지 시군구', '읍면동 (행정동)', 'lat', 'lon']]
future_demand = pd.merge(future_demand, coords, on=['사용본거지 시군구', '읍면동 (행정동)'], how='left')

# 결과 확인
future_demand.head()

서울특별시 서울특별시 강남구 강남구 개포1동  -> lat: None, lon: None
서울특별시 서울특별시 강남구 강남구 개포2동  -> lat: 37.482864637708445, lon: 127.07520286222606
서울특별시 서울특별시 강남구 강남구 개포4동  -> lat: 37.482864637708445, lon: 127.07520286222606
서울특별시 서울특별시 강남구 강남구 논현1동  -> lat: None, lon: None
서울특별시 서울특별시 강남구 강남구 논현2동  -> lat: 37.51588973774217, lon: 127.02023846047928
서울특별시 서울특별시 강남구 강남구 논현동  -> lat: 37.51588973774217, lon: 127.02023846047928
서울특별시 서울특별시 강남구 강남구 대치1동  -> lat: None, lon: None
서울특별시 서울특별시 강남구 강남구 대치2동  -> lat: None, lon: None
서울특별시 서울특별시 강남구 강남구 대치4동  -> lat: 37.4981828923879, lon: 127.07796227681962
서울특별시 서울특별시 강남구 강남구 도곡1동  -> lat: None, lon: None
서울특별시 서울특별시 강남구 강남구 도곡2동  -> lat: 37.492649668932174, lon: 127.0535135087495
서울특별시 서울특별시 강남구 강남구 삼성1동  -> lat: None, lon: None
서울특별시 서울특별시 강남구 강남구 삼성2동  -> lat: 37.516971034942344, lon: 127.04179187511397
서울특별시 서울특별시 강남구 강남구 세곡동  -> lat: 37.46703135655296, lon: 127.12147535716062
서울특별시 서울특별시 강남구 강남구 수서동  -> lat: 37.48693289059973, lon: 127.10854157896772
서울특별시 서울

,사용본거지 시군구,읍면동 (행정동),current_ev_count,total_vehicles,future_ev_count,adjusted_ev_demand,increase_amount,lat,lon
0,서울특별시 강남구,강남구 개포1동,31.0,3875.0,1937.5,1968.5,1937.5,NaN,NaN
1,서울특별시 강남구,강남구 개포2동,139.0,10266.0,5133.0,5272.0,5133.0,37.482865,127.075203
2,서울특별시 강남구,강남구 개포4동,79.0,8552.0,4276.0,4355.0,4276.0,37.482865,127.075203
3,서울특별시 강남구,강남구 논현1동,82.0,10388.0,5194.0,5276.0,5194.0,NaN,NaN
4,서울특별시 강남구,강남구 논현2동,111.0,10196.0,5098.0,5209.0,5098.0,37.515890,127.020238


In [26]:
# 1. 좌표가 없는 행 추출
missing_coords = future_demand[future_demand['lat'].isna() | future_demand['lon'].isna()].copy()
missing_coords['행정동만'] = missing_coords['읍면동 (행정동)'].str.split().str[1]
missing_coords['search_address'] = missing_coords['행정동만']
# 2. 좌표 요청 함수는 동일
def get_coords_vworld(address):
    apiurl = "https://api.vworld.kr/req/address?"
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "parcel",
        "key": VWORLD_API_KEY
    }
    response = requests.get(apiurl, params=params)
    if response.status_code == 200:
        try:
            point = response.json()['response']['result']['point']
            return float(point['y']), float(point['x'])  # 위도, 경도
        except:
            return None, None
    return None, None

# 3. 좌표 재요청
lat_filled, lon_filled = [], []
for addr in missing_coords['search_address']:
    lat, lon = get_coords_vworld(addr)
    lat_filled.append(lat)
    lon_filled.append(lon)
    print(f"{addr} -> lat: {lat}, lon: {lon}")
    time.sleep(0.3)

# 4. 결과 병합
missing_coords['lat'] = lat_filled
missing_coords['lon'] = lon_filled

future_demand = future_demand[~future_demand.index.isin(missing_coords.index)].copy()
future_demand = pd.concat([future_demand, missing_coords], ignore_index=True)

# 5. 아직 좌표가 채워지지 않은 행들 추출
retry_coords = missing_coords[missing_coords['lat'].isna() | missing_coords['lon'].isna()].copy()

# 6. 숫자 제거 ("개포1동" → "개포동")
import re
retry_coords['search_address'] = retry_coords['search_address'].apply(lambda x: re.sub(r'\d+', '', x))

# 7. 좌표 재요청 (숫자 제거된 주소로)
lat_retry, lon_retry = [], []
for addr in retry_coords['search_address']:
    lat, lon = get_coords_vworld(addr)
    lat_retry.append(lat)
    lon_retry.append(lon)
    print(f"[RETRY] {addr} -> lat: {lat}, lon: {lon}")
    time.sleep(0.3)

# 8. 재요청 결과 업데이트
retry_coords['lat'] = lat_retry
retry_coords['lon'] = lon_retry

# 기존 retry_coords 행 제거 후, 새 값으로 대체
future_demand = future_demand[~future_demand.index.isin(retry_coords.index)].copy()
future_demand = pd.concat([future_demand, retry_coords], ignore_index=True)

total_rows = len(future_demand)
missing_rows = future_demand['lat'].isna().sum()
print(f"📊 총 {total_rows}개 중 좌표가 채워진 행: {total_rows - missing_rows}개")
print(f"❗ 여전히 누락된 행: {missing_rows}개")

개포1동 -> lat: None, lon: None
논현1동 -> lat: None, lon: None
대치1동 -> lat: None, lon: None
대치2동 -> lat: None, lon: None
도곡1동 -> lat: None, lon: None
삼성1동 -> lat: None, lon: None
역삼1동 -> lat: None, lon: None
일원1동 -> lat: None, lon: None
고덕1동 -> lat: None, lon: None
고덕2동 -> lat: None, lon: None
길1동 -> lat: None, lon: None
둔촌1동 -> lat: None, lon: None
둔촌2동 -> lat: None, lon: None
명일1동 -> lat: None, lon: None
명일2동 -> lat: None, lon: None
상일제1동 -> lat: None, lon: None
상일제2동 -> lat: None, lon: None
성내1동 -> lat: 37.772578071173626, lon: 128.89979640245963
성내2동 -> lat: 37.18223068467941, lon: 128.97817986047033
성내3동 -> lat: 37.64392447904207, lon: 126.91160120572815
암사1동 -> lat: None, lon: None
암사2동 -> lat: None, lon: None
암사3동 -> lat: None, lon: None
천호1동 -> lat: None, lon: None
천호2동 -> lat: None, lon: None
천호3동 -> lat: None, lon: None
하일동 -> lat: None, lon: None
번1동 -> lat: None, lon: None
번2동 -> lat: None, lon: None
번3동 -> lat: None, lon: None
삼각산동 -> lat: None, lon: None
송중동 -> lat: None, lon:

In [32]:
future_demand_cleaned = future_demand.dropna(subset=['lat', 'lon']).copy()
future_demand_cleaned.to_csv(
    "../outputs/trash/future_demand.csv",
    index=False,
    encoding='utf-8-sig'
)

In [34]:
# grid_system 파일 로드
grid = pd.read_csv("../data/processed/grid_system_processed.csv")  # 반드시 min_lat, max_lat, min_lon, max_lon 포함

def find_grid_id(row):
    matched = grid[
        (grid['min_lat'] <= row['lat']) & (row['lat'] <= grid['max_lat']) &
        (grid['min_lon'] <= row['lon']) & (row['lon'] <= grid['max_lon'])
    ]
    return matched['grid_id'].values[0] if not matched.empty else None

# grid_id 매핑
future_demand_cleaned['grid_id'] = future_demand_cleaned.apply(find_grid_id, axis=1)

# 매핑되지 않은 행 제거 (보통 극히 일부)
future_demand_gridded = future_demand_cleaned.dropna(subset=['grid_id']).copy()

In [35]:
future_demand_gridded.to_csv(
    "../data/processed/future_demand_gridded.csv",
    index=False,
    encoding='utf-8-sig'
)

In [44]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# 1. 데이터 불러오기
future_df = pd.read_csv("../data/processed/future_demand_gridded.csv")
current_df = pd.read_csv("../data/processed/grid_features.csv")


# 2. grid_id 기준으로 병합
merged = pd.merge(
    current_df[['grid_id', 'demand_score']],
    future_df[['grid_id', 'adjusted_ev_demand']],
    on='grid_id',
    how='inner'  # 필요한 경우 outer도 가능
)

# 3. 가중 평균 계산
merged['log_future'] = np.log1p(merged['adjusted_ev_demand'])

scaler = MinMaxScaler()
merged['norm_current'] = scaler.fit_transform(merged[['demand_score']])
merged['norm_log_future'] = scaler.fit_transform(merged[['log_future']])

merged['final_score'] = 0.6 * merged['norm_current'] + 0.4 * merged['norm_log_future']


# 4. 결과 저장
merged.to_csv("../data/processed/future_demand_gridded_2.csv", index=False)

# XX 참고용 XX

In [40]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 로드
future_df = pd.read_csv("../data/processed/future_demand_gridded.csv")
current_df = pd.read_csv("../data/processed/grid_features.csv")

# 병합
merged = pd.merge(
    current_df[['grid_id', 'demand_score']],
    future_df[['grid_id', 'adjusted_ev_demand']],
    on='grid_id',
    how='inner'
)

# 요약 통계 출력
print("현재 수요 (demand_score):")
print(merged['demand_score'].describe())
print("\n미래 수요 (adjusted_ev_demand):")
print(merged['adjusted_ev_demand'].describe())


현재 수요 (demand_score):
count    236.000000
mean      48.141102
std       11.616603
min       22.050000
25%       43.220000
50%       48.130000
75%       55.620000
max       72.390000
Name: demand_score, dtype: float64

미래 수요 (adjusted_ev_demand):
count      236.000000
mean      3794.379237
std       2050.197915
min          0.000000
25%       2585.250000
50%       3392.750000
75%       4581.625000
max      14252.500000
Name: adjusted_ev_demand, dtype: float64
